Instalações e importações necessárias



In [203]:
!pip install pydantic
!pip install pydantic[email]

In [204]:
from datetime import datetime, timedelta
from enum import IntEnum
from typing import List, Optional, Dict, Any, ClassVar
from uuid import UUID, uuid4
import re
import json

from pydantic import (
    BaseModel,
    Field,
    EmailStr,
    field_validator,
    model_validator,
    field_serializer,
    ConfigDict,
    HttpUrl,
    RootModel
)

Criação de um REGEX para validar números ISBN

In [205]:
ISBN_REGEX = re.compile(
    r"^(97[89])?\d{1,5}[- ]?\d{1,7}[- ]?\d{1,7}[- ]?\d{1,7}[- ]?[0-9X]$"
)

Criação das classes BookStatus, que definem os possíveis estados de um livro na biblioteca e BookGenre, que são as categorias de gêneros de livros, cada um com um valor diferente

In [206]:
class BookStatus(IntEnum):
    AVAILABLE = 1
    BORROWED = 2
    MAINTENANCE = 3
    LOST = 4

class BookGenre(IntEnum):
    FICTION = 1
    NON_FICTION = 2
    SCIENCE = 3
    TECHNOLOGY = 4
    HISTORY = 5
    BIOGRAPHY = 6
    FANTASY = 7
    SCIENCE_FICTION = 8
    MYSTERY = 9
    THRILLER = 10
    ROMANCE = 11
    HORROR = 12
    OTHER = 99

Criação da classe Author, com características como id, nome, ano de nascimento, nacionalidade, biografia e website

o field_serializer converte o uuid para string quando o objeto é serializado para json

ja o field_validator verifica se o ano de nascimento não é um ano acima do atual

In [207]:
class Author(BaseModel):
    model_config = ConfigDict(extra="forbid")

    id: UUID = Field(default_factory=uuid4)
    name: str = Field(..., min_length=2, max_length=100)
    birth_year: Optional[int] = Field(None, ge=1000, le=datetime.now().year)
    nationality: Optional[str] = Field(None, min_length=2, max_length=50)
    biography: Optional[str] = Field(None, max_length=2000)
    website: Optional[HttpUrl] = None

    @field_serializer("id", when_used="json")
    def serialize_id(self, id: UUID) -> str:
        return str(id)

    @field_validator("birth_year")
    @classmethod
    def validate_birth_year(cls, value: Optional[int]) -> Optional[int]:
        if value is not None and value > datetime.now().year:
            raise ValueError("Birth year cannot be in the future")
        return value

Criação da Classe Book, tendo atributos como título, ISBN, autores, data de publicação, generos, páginas

e também o authors_map, uma variável de classe que funciona de maneira semelhante a um banco de dados

o field_serializer converte tanto uuid individuais tanto listas de uuids para string quando o objeto é serializado para json

o field_validator isbn verifica se o ISBN está em um formato válido

o model_validator after verifica se todos os autores referenciados no livro existem no authors_map (especie de banco de dados)

e o get_authors retorna os objetos author completos a partir dos ids armazenados

In [208]:
class Book(BaseModel):
    model_config = ConfigDict(extra="forbid")

    id: UUID = Field(default_factory=uuid4)
    title: str = Field(..., min_length=1, max_length=200)
    isbn: str = Field(..., description="ISBN-10 or ISBN-13 format")
    authors: List[UUID] = Field(..., min_items=1, description="List of author IDs")
    publication_year: int = Field(..., ge=1000, le=datetime.now().year + 2)
    genres: List[BookGenre] = Field(..., min_items=1, max_items=3)
    description: Optional[str] = Field(None, max_length=5000)
    pages: Optional[int] = Field(None, gt=0)
    status: BookStatus = Field(default=BookStatus.AVAILABLE)
    added_at: datetime = Field(default_factory=datetime.now)

    _authors_map: ClassVar[Dict[UUID, Author]] = {}

    @field_serializer("id", "authors", when_used="json")
    def serialize_uuid(self, v: UUID | List[UUID]) -> str | List[str]:
        if isinstance(v, list):
            return [str(item) for item in v]
        return str(v)

    @field_validator("isbn")
    @classmethod
    def validate_isbn(cls, v: str) -> str:
        clean_isbn = re.sub(r"[- ]", "", v)
        if not ISBN_REGEX.match(v):
            raise ValueError("Invalid ISBN format")
        return clean_isbn

    @model_validator(mode="after")
    def validate_authors_exist(self) -> "Book":
        for author_id in self.authors:
            if author_id not in self._authors_map:
                raise ValueError(f"Author with ID {author_id} does not exist")
        return self

    def get_authors(self) -> List[Author]:
        return [self._authors_map[author_id] for author_id in self.authors]

criação da classe LibraryUser, que representa um usuário da biblioteca, possuindo email, um número de membro e se a conta está ativa ou não

enquanto o field_serializer converte uuid para string

o model_validator before define automaticamente a data de validade da associação para 1 ano após a data de ingresso caso não seja fornecida

enquanto o model_validator after garante que a data de expiração não seja anterior a data de ingresso

In [209]:
class LibraryUser(BaseModel):
    model_config = ConfigDict(extra="forbid")

    id: UUID = Field(default_factory=uuid4)
    name: str = Field(..., min_length=2, max_length=100)
    email: EmailStr = Field(...)
    membership_number: str = Field(..., pattern=r"^LIB-\d{6}$")
    joined_date: datetime = Field(default_factory=datetime.now)
    membership_valid_until: datetime = Field(...)
    is_active: bool = Field(default=True)

    @field_serializer("id", when_used="json")
    def serialize_id(self, id: UUID) -> str:
        return str(id)

    @model_validator(mode="before")
    @classmethod
    def set_membership_period(cls, data: Dict[str, Any]) -> Dict[str, Any]:
        if "membership_valid_until" not in data and "joined_date" in data:
            data["membership_valid_until"] = data["joined_date"] + timedelta(days=365)
        elif "membership_valid_until" not in data:
            joined = datetime.now()
            data["joined_date"] = joined
            data["membership_valid_until"] = joined + timedelta(days=365)
        return data

    @model_validator(mode="after")
    def validate_dates(self) -> "LibraryUser":
        if self.membership_valid_until < self.joined_date:
            raise ValueError("Membership expiration cannot be earlier than join date")
        return self

criação da classe BookLoan, que funciona como um empréstimo de livro feito pela biblioteca, contendo ids do livro e do usuário, data de empréstimo e devolução, se está atrasado ou não e também um mapa para vermos livros e usuários disponíveis

enquanto o field_serializer converte uuids para strings durante a serialização

o modelo_validator before define a data de devolução para 14 dias após o empréstimo caso não for fornecida

enquanto o modelo_validator after verifica muitas coisas para definir o empréstimo como válido, como por exemplo se o livro e usuário existem, se o livro está disponível e o usuário ativo sem assosciação expirada e também se as datas são válidas, além do status de atraso

e o def_complete_return marca um livro como devolvido e atualiza o seu status

In [210]:
class BookLoan(BaseModel):
    model_config = ConfigDict(extra="forbid")

    id: UUID = Field(default_factory=uuid4)
    book_id: UUID = Field(...)
    user_id: UUID = Field(...)
    loan_date: datetime = Field(default_factory=datetime.now)
    due_date: datetime = Field(...)
    return_date: Optional[datetime] = None
    is_overdue: bool = Field(default=False)

    _books_map: ClassVar[Dict[UUID, Book]] = {}
    _users_map: ClassVar[Dict[UUID, LibraryUser]] = {}

    @field_serializer("id", "book_id", "user_id", when_used="json")
    def serialize_uuid(self, v: UUID) -> str:
        return str(v)

    @model_validator(mode="before")
    @classmethod
    def set_due_date(cls, data: Dict[str, Any]) -> Dict[str, Any]:
        if "due_date" not in data and "loan_date" in data:
            data["due_date"] = data["loan_date"] + timedelta(days=14)
        elif "due_date" not in data:
            data["loan_date"] = datetime.now()
            data["due_date"] = data["loan_date"] + timedelta(days=14)
        return data

    @model_validator(mode="after")
    def validate_loan(self) -> "BookLoan":
        if self.book_id not in self._books_map:
            raise ValueError(f"Book with ID {self.book_id} does not exist")
        if self.user_id not in self._users_map:
            raise ValueError(f"User with ID {self.user_id} does not exist")

        book = self._books_map[self.book_id]
        if book.status != BookStatus.AVAILABLE and self.return_date is None:
            raise ValueError(f"Book is not available for loan (current status: {book.status.name})")

        user = self._users_map[self.user_id]
        if not user.is_active:
            raise ValueError(f"User {user.name} is not active")
        if user.membership_valid_until < datetime.now():
            raise ValueError(f"User {user.name}'s membership has expired")

        if self.due_date < self.loan_date:
            raise ValueError("Due date cannot be earlier than loan date")
        if self.return_date and self.return_date < self.loan_date:
            raise ValueError("Return date cannot be earlier than loan date")

        if self.return_date is None and datetime.now() > self.due_date:
            self.is_overdue = True

        return self

    def complete_return(self) -> None:
        self.return_date = datetime.now()
        self.is_overdue = False

        book = self._books_map.get(self.book_id)
        if book:
            book.status = BookStatus.AVAILABLE

hora dos testes

comentarei ao decorrer do código e não aqui em cima como de costume pois devido ao tamanho do código ficará de melhor entendimento os comentários ao decorrer das células

In [211]:
def test_library_system():
  #criação de dois autores
    author1 = Author(
        name="Rick Riordan",
        birth_year=1964,
        nationality="North-American",
        biography="American author and schoolteacher"
    )
    author2 = Author(
        name="J.K. Rowling",
        birth_year=1965,
        nationality="British",
        website="https://www.jkrowling.com"
    )
    #adicionando os autores no "banco de dados"
    Book._authors_map[author1.id] = author1
    Book._authors_map[author2.id] = author2

  #criação de dois livros, cada um de um autor diferente
    book1 = Book(
        title="The Lightning Thief",
        isbn="978-0786838653",
        authors=[author1.id],
        publication_year=2005,
        description="The Lightning Thief follows the story of young Percy Jackson, a troubled 12-year-old boy with a secret unknown even to himself",
        genres=[BookGenre.SCIENCE_FICTION], #"mudei" o genero desse para não ficar igual ao do segundo livro
        pages=384
    )

    book2 = Book(
        title="Harry Potter and the Philosopher's Stone",
        isbn="978-0747532699",
        authors=[author2.id],
        publication_year=1997,
        genres=[BookGenre.FANTASY, BookGenre.FICTION],
        pages=223
    )

    #adicionando os livros ao "banco de dados"
    BookLoan._books_map[book1.id] = book1
    BookLoan._books_map[book2.id] = book2

  #criação de dois usuários
    user1 = LibraryUser(
        name="Enzo Rodrigues",
        email="enzo.rodrigues@example.com",
        membership_number="LIB-123456",
        joined_date=datetime.now() - timedelta(days=30)
    )

    user2 = LibraryUser(
        name="Thiago Naves",
        email="thiago.naves@example.com",
        membership_number="LIB-654321",
        joined_date=datetime.now() - timedelta(days=60),
        membership_valid_until=datetime.now() - timedelta(days=10)  #associação expirada para fim de testes
    )

    #adicionando os usuários ao "banco de dados"
    BookLoan._users_map[user1.id] = user1
    BookLoan._users_map[user2.id] = user2

    #criando empréstimos
    loan1 = BookLoan(
        book_id=book1.id,
        user_id=user1.id
    )

    #esse empréstimo tem que falhar porque a associação desse usuário expirou
    try:
        loan2 = BookLoan(
            book_id=book2.id,
            user_id=user2.id
        )
        print("ERROR: Loan to user with expired membership was created!")
    except ValueError as e:
        print(f"Correct error raised: {e}")

    #atualizar status do livro emprestado
    book1.status = BookStatus.BORROWED

    #tentando emprestar um livro que já está emprestado
    try:
        loan3 = BookLoan(
            book_id=book1.id,
            user_id=user1.id
        )
        print("ERROR: Loan for an already borrowed book was created!")
    except ValueError as e:
        print(f"Correct error raised: {e}")

    #devpolver livro
    loan1.complete_return()
    print(f"Book '{book1.title}' returned successfully. Status: {book1.status.name}")

    #emprestando o livro novamente
    book1.status = BookStatus.AVAILABLE  #funciona como um tipo de reset de status
    loan4 = BookLoan(
        book_id=book1.id,
        user_id=user1.id,
        due_date=datetime.now() + timedelta(days=7)  #prazo de empréstimo "personalizado"
    )
    print(f"Book '{book1.title}' borrowed again until {loan4.due_date}")

    #serializando para JSON
    class BookList(RootModel):
        root: List[Book]

    books = BookList(root=[book1, book2])
    books_json = books.model_dump_json(indent=2)
    print("\nBooks JSON Serialization:")
    print(books_json)

    #validação de ISBN inválido
    try:
        invalid_book = Book(
            title="Invalid Book",
            isbn="123-456",  #ISBN inválido
            authors=[author1.id],
            publication_year=2023,
            genres=[BookGenre.FICTION]
        )
        print("ERROR: Book with invalid ISBN was created!")
    except ValueError as e:
        print(f"Correct error raised: {e}")

"explicarei" o output no relatório desse card

In [212]:
if __name__ == "__main__":
    test_library_system()

Correct error raised: 1 validation error for BookLoan
  Value error, User Thiago Naves's membership has expired [type=value_error, input_value={'book_id': UUID('6f65a21...26, 11, 57, 41, 326062)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error
Correct error raised: 1 validation error for BookLoan
  Value error, Book is not available for loan (current status: BORROWED) [type=value_error, input_value={'book_id': UUID('cea704b...26, 11, 57, 41, 326234)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error
Book 'The Lightning Thief' returned successfully. Status: AVAILABLE
Book 'The Lightning Thief' borrowed again until 2025-03-19 11:57:41.326409

Books JSON Serialization:
[
  {
    "id": "cea704b4-9e6a-4254-8814-e94b7b8b864a",
    "title": "The Lightning Thief",
    "isbn": "9780786838653",
    "authors": [
      "f4d2fca4-49ac-44f6-8fb5-05471ee860f3"
    ],
    "publication_year": 2005,
    "